# Example 7: Multi-Handler Logging - Complete Solution

## What You'll Learn

This notebook demonstrates using multiple handlers simultaneously:

- **Multiple Handlers** - Console + File + Cloud
- **Production Setup** - Real-world logging configuration
- **Different Outputs** - Each handler with different format
- **Complete Workflow** - From development to production
- **Best Practices** - Optimal logging strategy


## Setup - Imports


In [ ]:
import asyncio
import os
from dc_logger.client.Log import LogEntry, LogLevel
from dc_logger.client.base import Logger, HandlerInstance
from dc_logger.services.console.base import ConsoleHandler, Console_ServiceConfig
from dc_logger.logs.services.file import FileHandler, File_ServiceConfig

# Create logs directory
os.makedirs("multi_handler_logs", exist_ok=True)

print("All imports successful!")
print("Logs directory: multi_handler_logs/")


---
## Part 1: Logger with Console + File Handlers


In [ ]:
print("=" * 70)
print("Console + File Logging")
print("=" * 70)

# Create console handler (text format for readability)
console_config = Console_ServiceConfig(
    service_name="console",
    output_mode="console"
)
console_handler = ConsoleHandler(config=console_config)

# Create file handler (JSON format for detailed logs)
file_config = File_ServiceConfig(
    service_name="file",
    destination="multi_handler_logs/app.json",
    format="json",
    append=True
)
file_handler = FileHandler(service_config=file_config)

# Create logger with BOTH handlers
multi_logger = Logger(
    app_name="multi_handler_app",
    handlers=[
        HandlerInstance(handler=console_handler, output_type="text"),
        HandlerInstance(handler=file_handler, output_type="json")
    ]
)

print("\nLogger created with 2 handlers:")
print("  1. Console (text format)")
print("  2. File (JSON format)")
print("\nLogs will appear in console AND be saved to file\n")
print("-" * 70)

# Log some events
await multi_logger.info("Application started")
await multi_logger.info(
    "User logged in",
    user_id="user_alice",
    tenant_id="tenant_acme"
)
await multi_logger.warning("Cache approaching limit")

print("-" * 70)
print(f"\nLogs also saved to: {file_config.destination}")


---
## Part 2: Multiple File Formats Simultaneously


In [ ]:
print("=" * 70)
print("Multiple File Formats: JSON + Text + CSV")
print("=" * 70)

# Create three file handlers with different formats
json_file_config = File_ServiceConfig(
    service_name="json_file",
    destination="multi_handler_logs/detailed.json",
    format="json",
    append=True
)

text_file_config = File_ServiceConfig(
    service_name="text_file",
    destination="multi_handler_logs/quick_scan.log",
    format="text",
    append=True
)

csv_file_config = File_ServiceConfig(
    service_name="csv_file",
    destination="multi_handler_logs/spreadsheet.csv",
    format="csv",
    append=True
)

# Create handlers
json_handler = FileHandler(service_config=json_file_config)
text_handler = FileHandler(service_config=text_file_config)
csv_handler = FileHandler(service_config=csv_file_config)

# Create logger with all three
format_logger = Logger(
    app_name="format_demo",
    handlers=[
        HandlerInstance(handler=json_handler, output_type="json"),
        HandlerInstance(handler=text_handler, output_type="text"),
        HandlerInstance(handler=csv_handler, output_type="csv")
    ]
)

print("\nLogger created with 3 file handlers:")
print("  1. JSON for analysis tools")
print("  2. Text for grep/tail")
print("  3. CSV for Excel")
print("\nLogging events...\n")

await format_logger.info("Service initialized")
await format_logger.info("Processing batch", extra={"items": 100})
await format_logger.warning("Slow query detected", duration_ms=2500)
await format_logger.error("Connection timeout", extra={"host": "db.example.com"})

print("\nLogs written to:")
print(f"  JSON: {json_file_config.destination}")
print(f"  Text: {text_file_config.destination}")
print(f"  CSV: {csv_file_config.destination}")


In [ ]:
print("=" * 70)
print("Production Setup: Console + JSON File + Text File")
print("=" * 70)

# Production-style setup
prod_console_config = Console_ServiceConfig(
    service_name="prod_console",
    output_mode="console"
)

prod_json_config = File_ServiceConfig(
    service_name="prod_json",
    destination="multi_handler_logs/production/app.json",
    format="json",
    append=True
)

prod_text_config = File_ServiceConfig(
    service_name="prod_text",
    destination="multi_handler_logs/production/app.log",
    format="text",
    append=True
)

# Create handlers
prod_console = ConsoleHandler(config=prod_console_config)
prod_json = FileHandler(service_config=prod_json_config)
prod_text = FileHandler(service_config=prod_text_config)

# Create production logger
production_logger = Logger(
    app_name="production_app",
    handlers=[
        HandlerInstance(handler=prod_console, output_type="text"),
        HandlerInstance(handler=prod_json, output_type="json"),
        HandlerInstance(handler=prod_text, output_type="text")
    ]
)

print("\nProduction logger with 3 handlers:")
print("  1. Console - Real-time monitoring during deployment")
print("  2. JSON File - Detailed logs for analysis")
print("  3. Text File - Quick scanning with grep/tail")
print("  (4. Cloud - Would be Datadog/CloudWatch in real production)")
print("\nSimulating production events...\n")
print("-" * 70)

# Simulate production events
await production_logger.info(
    "Application started",
    extra={"version": "2.1.0", "environment": "production"}
)

await production_logger.info(
    "Database pool initialized",
    extra={"connections": 20, "max": 50}
)

await production_logger.info(
    "User request",
    user_id="user_bob_456",
    tenant_id="tenant_startup",
    method="POST",
    url="/api/projects",
    status_code=201,
    duration_ms=145
)

await production_logger.warning(
    "Cache miss rate high",
    extra={"miss_rate": 0.35, "threshold": 0.20}
)

await production_logger.error(
    "External API timeout",
    method="GET",
    url="https://external-api.example.com/data",
    status_code=504,
    duration_ms=5000,
    extra={"retry_count": 3}
)

print("-" * 70)
print("\nLogs written to:")
print(f"  Console (above)")
print(f"  JSON: {prod_json_config.destination}")
print(f"  Text: {prod_text_config.destination}")


In [ ]:
print("=" * 70)
print("Environment-Specific Logging Strategy")
print("=" * 70)

def create_logger_for_environment(env: str) -> Logger:
    """Create logger with handlers appropriate for environment."""
    
    handlers = []
    
    if env == "development":
        # Dev: Console only for fast feedback
        console_config = Console_ServiceConfig(
            service_name=f"console_{env}",
            output_mode="console"
        )
        console_handler = ConsoleHandler(config=console_config)
        handlers.append(
            HandlerInstance(handler=console_handler, output_type="text")
        )
        
    elif env == "staging":
        # Staging: Console + JSON file
        console_config = Console_ServiceConfig(
            service_name=f"console_{env}",
            output_mode="console"
        )
        file_config = File_ServiceConfig(
            service_name=f"file_{env}",
            destination=f"multi_handler_logs/{env}/app.json",
            format="json",
            append=True
        )
        console_handler = ConsoleHandler(config=console_config)
        file_handler = FileHandler(service_config=file_config)
        
        handlers.extend([
            HandlerInstance(handler=console_handler, output_type="text"),
            HandlerInstance(handler=file_handler, output_type="json")
        ])
        
    elif env == "production":
        # Production: JSON file + Text file + Cloud
        json_config = File_ServiceConfig(
            service_name=f"json_{env}",
            destination=f"multi_handler_logs/{env}/app.json",
            format="json",
            append=True
        )
        text_config = File_ServiceConfig(
            service_name=f"text_{env}",
            destination=f"multi_handler_logs/{env}/app.log",
            format="text",
            append=True
        )
        
        json_handler = FileHandler(service_config=json_config)
        text_handler = FileHandler(service_config=text_config)
        
        handlers.extend([
            HandlerInstance(handler=json_handler, output_type="json"),
            HandlerInstance(handler=text_handler, output_type="text")
        ])
        # Would add cloud handler here in real production
    
    return Logger(app_name=f"app_{env}", handlers=handlers)

# Create loggers for each environment
environments = ["development", "staging", "production"]

print("\nLogging strategy by environment:\n")
for env in environments:
    logger = create_logger_for_environment(env)
    num_handlers = len(logger.handlers)
    
    print(f"{env.upper():12} - {num_handlers} handler(s)")
    if env == "development":
        print("             Console only (fast feedback)")
    elif env == "staging":
        print("             Console + JSON file (debugging + persistence)")
    elif env == "production":
        print("             JSON + Text files (+ Cloud in real setup)")
    print()

# Test production logger
print("Testing production logger:")
print("-" * 70)
prod_logger = create_logger_for_environment("production")
await prod_logger.info("Test log from production logger")
print("-" * 70)


---
## Part 5: Verify All Logs Were Written


In [ ]:
print("=" * 70)
print("Verify All Log Files Created")
print("=" * 70)

print("\nAll log files created:\n")
for root, dirs, files in os.walk("multi_handler_logs"):
    level = root.replace("multi_handler_logs", "").count(os.sep)
    indent = " " * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = " " * 2 * (level + 1)
    for file in files:
        filepath = os.path.join(root, file)
        size = os.path.getsize(filepath)
        print(f"{sub_indent}{file} ({size} bytes)")

# Show sample content from one file
print("\n\nSample from production JSON log:")
print("-" * 70)
try:
    with open("multi_handler_logs/production/app.json", "r") as f:
        first_line = f.readline()
        if first_line:
            import json
            log_entry = json.loads(first_line)
            print(f"Level: {log_entry.get('level')}")
            print(f"Message: {log_entry.get('message')}")
            print(f"Timestamp: {log_entry.get('timestamp')}")
except FileNotFoundError:
    print("File not found (expected if running without all examples)")


---
## Summary

### What We Learned

1. **Multiple Handlers** - Use console, file, and cloud handlers together
2. **Different Formats** - Each handler can use different format (JSON/Text/CSV)
3. **Environment Strategy** - Different handlers for dev/staging/production
4. **Production Setup** - Best practices for real applications
5. **Flexibility** - Mix and match handlers based on needs

### Handler Combinations

Common combinations by use case:

**Development**
- Console (text) - Fast feedback

**Staging**
- Console (text) - Real-time monitoring
- JSON file - Debugging

**Production**
- JSON file - Detailed logs for analysis
- Text file - Quick scanning
- Cloud (Datadog) - Centralized monitoring and alerts

### Key Takeaways

- **One Logger, Many Outputs** - Single log call writes to all handlers
- **No Extra Code** - Same logging code works across environments
- **Format Flexibility** - Choose best format for each destination
- **Easy Configuration** - Just add/remove handlers
- **Environment Aware** - Different handlers per environment

### Benefits of Multi-Handler Logging

1. **Redundancy** - If cloud fails, still have file logs
2. **Different Audiences** - Console for devs, cloud for ops
3. **Cost Optimization** - Local files for debug, cloud for critical
4. **Flexibility** - Easy to add/remove handlers without code changes

### Production Checklist

- [ ] Console handler for deployment monitoring
- [ ] JSON file handler for detailed logs
- [ ] Text file handler for quick grep/tail
- [ ] Cloud handler (Datadog) for alerts and dashboards
- [ ] Log rotation configured for file handlers
- [ ] Environment-specific handler selection
- [ ] Secure storage for log files
- [ ] Backup strategy for critical logs

### Next Steps

You now have complete knowledge of the dc_logger library:

1. **Basic Logs** - LogEntry, LogLevel, context objects
2. **Extractors** - Automatic context extraction
3. **Decorators** - Automatic function logging
4. **Console Logging** - Real-time output
5. **File Logging** - Persistent storage (JSON/Text/CSV)
6. **Cloud Logging** - Datadog integration
7. **Multi-Handler** - Complete production setup

Start logging!
